In [1]:
import pandas as pd

In [8]:
df = pd.read_parquet('../../data/curated/2023_prob.parquet')

In [9]:
df

,playerid,match_id,player_first_name,player_last_name,team_name,0,1,2,3
0,CD_I1000223,10007,Liam,Baker,Richmond,0.997965,0.001543,0.000450,0.000041
1,CD_I1002245,10007,Noah,Balta,Richmond,0.999350,0.000532,0.000110,0.000008
2,CD_I993993,10007,Shai,Bolton,Richmond,0.967135,0.021222,0.009677,0.001966
3,CD_I295203,10007,Nathan,Broad,Richmond,0.997749,0.001720,0.000480,0.000051
4,CD_I270896,10007,Trent,Cotchin,Richmond,0.995610,0.003199,0.001077,0.000114
...,...,...,...,...,...,...,...,...,...
9517,CD_I294305,10199,Lachie,Whitfield,GWS Giants,0.636328,0.140450,0.147469,0.075753
9518,CD_I1008139,10199,Toby,Bedford,GWS Giants,0.994316,0.003926,0.001569,0.000189
9519,CD_I290675,10199,Daniel,Lloyd,GWS Giants,0.991018,0.006258,0.002408,0.000316
9520,CD_I1008123,10199,Jake,Riccardi,GWS Giants,0.997230,0.002118,0.000588,0.000063


In [10]:
match_ids = df.match_id.unique()

In [14]:
def standardise_votes(df):
    
    ini_sum_1 = sum(df['1'])
    ini_sum_2 = sum(df['2'])
    ini_sum_3 = sum(df['3'])
    
    df['new_3'] = df['3'] / ini_sum_3
    df['new_2'] = df['2'] / ini_sum_2
    df['new_1'] = df['1'] / ini_sum_1

    df['new_0'] = 1 - (df['new_1'] + df['new_2'] + df['new_3'])
    
    df['exp_votes'] = 0*df['new_0'] + 1*df['new_1'] + 2*df['new_2'] + 3*df['new_3']
    
    
    return df[['playerid', 'match_id', 'player_first_name', 'player_last_name', 'team_name', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes']]

In [15]:
final_df = pd.DataFrame(columns=['playerid', 'match_id', 'player_first_name', 'player_last_name', 'team_name', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes'])


for idx in match_ids:
    
    df_match = df[df['match_id'] == idx].reset_index() 
    
    final_df = pd.concat([final_df, standardise_votes(df_match)])

In [16]:
final_df

,playerid,match_id,player_first_name,player_last_name,team_name,new_0,new_1,new_2,new_3,exp_votes
0,CD_I1000223,10007,Liam,Baker,Richmond,0.997575,0.001729,0.000598,0.000098,0.003219
1,CD_I1002245,10007,Noah,Balta,Richmond,0.999239,0.000596,0.000146,0.000019,0.000946
2,CD_I993993,10007,Shai,Bolton,Richmond,0.958720,0.023773,0.012841,0.004665,0.063452
3,CD_I295203,10007,Nathan,Broad,Richmond,0.997315,0.001926,0.000637,0.000122,0.003567
4,CD_I270896,10007,Trent,Cotchin,Richmond,0.994717,0.003584,0.001429,0.000270,0.007253
...,...,...,...,...,...,...,...,...,...,...
41,CD_I294305,10199,Lachie,Whitfield,GWS Giants,0.516620,0.149933,0.182604,0.150843,0.967670
42,CD_I1008139,10199,Toby,Bedford,GWS Giants,0.993490,0.004191,0.001943,0.000376,0.009205
43,CD_I290675,10199,Daniel,Lloyd,GWS Giants,0.989708,0.006681,0.002981,0.000630,0.014533
44,CD_I1008123,10199,Jake,Riccardi,GWS Giants,0.996884,0.002262,0.000728,0.000126,0.004097


In [17]:
final_df.to_parquet('../../data/curated/adjusted_probabilities_2023.parquet')